In [1]:
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

     |████████████████████████████████| 165.1MB 75kB/s  eta 0:00:01    |████▊                           | 24.2MB 537kB/s eta 0:04:23     |█████▏                          | 26.8MB 1.3MB/s eta 0:01:45     |██████████████▍                 | 74.1MB 878kB/s eta 0:01:44
     |████████████████████████████████| 61kB 1.7MB/s eta 0:00:01
     |████████████████████████████████| 460kB 1.5MB/s eta 0:00:01
     |████████████████████████████████| 6.0MB 2.0MB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 841kB/s eta 0:00:01     |███████████▍                    | 1.1MB 1.7MB/s eta 0:00:02
  Using cached https://files.pythonhosted.org/packages/79/4c/7c3275a01e12ef9368a892926ab932b33bb13d55794881e3573482b378a7/Keras_Preprocessing-1.1.2-py2.py3-none-any.whl
     |████████████████████████████████| 61kB 1.0MB/s eta 0:00:01
     |████████████████████████████████| 71kB 1.0MB/s eta 0:00:01
     |████████████████████████████████| 2.8MB 1.1MB/s eta 0:00:01
     |████████████████████████████████| 14

  Using cached https://files.pythonhosted.org/packages/05/57/ce2e7a8fa7c0afb54a0581b14a65b56e62b5759dbc98e80627142b8a3704/oauthlib-3.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/62/1e/a94a8d635fa3ce4cfc7f506003548d0a2447ae76fd5ca53932970fe3053f/pyasn1-0.4.8-py2.py3-none-any.whl
ERROR: tensorboard 2.5.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.23.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.13.0
    Uninstalling tensorflow-estimator-1.13.0:
      Successfully uninstalled tensorflow-estimator-1.13.0
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1
  Found existing installation: h5py 2.9.0
    Uninstalling h5py-2.9.0:
      Successfully uninstalled h5py-2.9.0
  Found existing installation: Keras-Preprocessing 1.1.0
    Uninstalling Keras-Preprocessing-1.1.0:
      Successfully uninstalled Keras-Preprocessing-1.1.0
  Fo

In [7]:
pip install gym[atari]

     |████████████████████████████████| 3.5MB 608kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import gym 
import random

In [5]:
env = gym.make('SpaceInvaders-v0') # create gym environment 
height, width, channels = env.observation_space.shape #this gives an image shape.
actions = env.action_space.n  #gives number of actions that we can take at Action Space

In [6]:
env.unwrapped.get_action_meanings() # showing action space meaning. NOOP: no action.


['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [5]:
episodes = 5 # 5 bölüm random oyun sonuçları
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5]) # actionları random olarak aldığını varsayalım.
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:110.0
Episode:2 Score:120.0
Episode:3 Score:210.0
Episode:4 Score:155.0
Episode:5 Score:130.0


In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [7]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [14]:
del model

In [15]:
model = build_model(height, width, channels, actions)

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 6)                

In [17]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [18]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [19]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [20]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)


Training for 10000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
  394/10000: episode: 1, duration: 11.155s, episode steps: 394, steps per second:  35, episode reward: 110.000, mean reward:  0.279 [ 0.000, 30.000], mean action: 2.437 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 1057/10000: episode: 2, duration: 86.881s, episode steps: 663, steps per second:   8, episode reward: 125.000, mean reward:  0.189 [ 0.000, 30.000], mean action: 2.468 [0.000, 5.000],  loss: 92.863172, mean_q: 10.324517, mean_eps: 0.907435
 1861/10000: episode: 3, duration: 921.332s, episode steps: 804, steps per second:   1, episode reward: 215.000, mean reward:  0.267 [ 0.000, 30.000], mean action: 2.409 [0.000, 5.000],  loss: 1.714442, mean_q: 5.251053, mean_eps: 0.868735
 2632/10000: episode: 4, duration: 855.500s, episode steps: 771, steps per second:   1, episode reward: 210.000, mean reward:  0.272 [ 0.000, 30.000], m

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))